# Handwritten Digit Recognition with Machine Learning

Just attempting to try handwritten digit recognition with machine learning.

Project inspired by scikit-learn's "Recognizing hand-written digits": https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html#sphx-glr-auto-examples-classification-plot-digits-classification-py

## Import libraries

In [47]:
import numpy as np
from numpy import asarray
import os
import time

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import tkinter as tk
from PIL import Image
from PIL import ImageGrab

## Import the images and labels. 

In [48]:
train_images = np.load('trainImages.npy')
train_labels = np.load('trainLabels.npy')

## Training the models.

In [49]:
#----------Random Forest----------
rf_model = RandomForestClassifier(n_estimators = 100, random_state = 1)
rf_model.fit(train_images, train_labels)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [54]:
#----------K-Nearest Neighbours----------
knn_model = KNeighborsClassifier(n_neighbors = 10, weights = "distance")
knn_model.fit(train_images, train_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='distance')

In [51]:
#----------Support Vector Machine (SVM)----------
svm_model = SVC(kernel = "rbf", C = 5, random_state = 1)
svm_model.fit(train_images, train_labels)

SVC(C=5, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
    verbose=False)

## GUI For Canvas 

<b>NOTE:</b> BEFORE RUNNING THE CODE, go on Display Settings -> Scale and layout -> then "Change the size of text, apps and other items" to 100% (for Windows 10). Otherwise, fetching the co-ordinates of the canvas will be inaccurate.

In [83]:
canvasWidth = 280
canvasHeight = 280
borderThickness = 2
mousePos = { "x":0, "y":0 }
lineWidth = 15
save_directory = "Data/"

# Sets the current position of the mouse.
def setPosition(event):
    mousePos["x"] = event.x 
    mousePos["y"] = event.y

# Draw on the canvas based on the current position of the mouse.
def draw(event):
    currX = mousePos["x"]
    currY = mousePos["y"]
    canvas.create_oval(currX, currY, currX, currY, width = lineWidth)
    setPosition(event)

# Takes a screenshot of the canvas and returns it.
def getCanvasScreenshot():
    canvas.update()
    
    # Added (borderThickness) to prevent the border from being part of the screenshot.
    x0 = canvas.winfo_rootx() + borderThickness
    y0 = canvas.winfo_rooty() + borderThickness
    x1 = x0 + canvasWidth
    y1 = y0 + canvasHeight
    
    screenShot = ImageGrab.grab((x0, y0, x1, y1))
    return screenShot

# Displays the window at the centre of the screen.
def centreWindow(width, height, window):
    window_width = width
    window_height = height
    x_window_pos = (window.winfo_screenwidth() / 2) - (window_width / 2)
    y_window_pos = (window.winfo_screenheight() / 2) - (window_height / 2)
    window.geometry("%dx%d+%d+%d" % (window_width, window_height, x_window_pos, y_window_pos))
    
# Makes the predictions on the image.
def predictImage():
    # Get screenshot and adjust the image.
    screenShot = getCanvasScreenshot()
    screenShot = screenShot.convert('P', palette = Image.ADAPTIVE, colors = 2)
    
    # Convert the image into a numpy array.
    npImg = asarray(screenShot, dtype = "int").reshape(1, -1)
    
    rf_pred = rf_model.predict(npImg)
    knn_pred = knn_model.predict(npImg)
    svm_pred = svm_model.predict(npImg)
    
    # Display the predictions on the GUI.
    lblRandForest.config(text = "Random Forest: " + str(rf_pred))
    lblKNN.config(text = "K-Nearest Neighbours: " + str(knn_pred))
    lblSVM.config(text = "SVM: " + str(svm_pred))
    btnWrongPred.pack() # Show the button.

# Displays a seperate window that allows the user to submit what was drawn
# if the prediction(s) was wrong.
def openPredictWindow():
    predictWindow = tk.Toplevel(root)
    window_width = 200
    window_height = 100
    centreWindow(window_width, window_height, predictWindow)
    predictWindow.resizable(0, 0)
    predictWindow.grab_set() # Prevent interaction of the main window.
    
    lblPredictPrompt = tk.Label(predictWindow, text = "Enter the number you drew:").pack()
    
    txtPredict = tk.Entry(predictWindow)
    txtPredict.pack()
    
    lblInvalidInput = tk.Label(predictWindow)
    lblInvalidInput.pack()
    
    # Lambda prevents the function from running (when opening the window).
    btnSubmit = tk.Button(predictWindow, text = "Submit", 
                          command = lambda : saveImage(txtPredict.get(), predictWindow, lblInvalidInput))
    btnSubmit.pack()

# Saves the canvas contents as a new training image, then closes the sub-window.
def saveImage(userPred, predictWindow, lblInvalidInput):
    # Remove whitespace (it's a single digit input)
    userPred = userPred.replace(" ", "") 
    
    # The input must be a single digit.
    if (not userPred.isnumeric()) or (len(userPred) > 1):
        lblInvalidInput.config(foreground = "red", text = "Invalid Input")
    else:
        # Close the window and wait (for it to close).
        # This avoids the window from being in the screenshot (if it is).
        predictWindow.destroy()
        time.sleep(1)
        
        # Save the image (name is based on number of files in the respective directory).
        path, dirs, files = next(os.walk(save_directory + userPred))
        file_count = len(files)
        screenShot = getCanvasScreenshot()
        screenShot.save(save_directory + userPred + "/" + str(file_count + 1) + ").png")
    
# Create the main window and add its elements.
root = tk.Tk()
window_width = 500
window_height = 500
centreWindow(window_width, window_height, root)
root.resizable(0, 0)

canvas = tk.Canvas(root, bg = "white", width = canvasWidth, height = canvasHeight, 
                   highlightthickness = borderThickness, highlightbackground = "black")
canvas.bind("<Button-1>", setPosition)
canvas.bind("<B1-Motion>", draw)
canvas.pack()

# Create and display each button (apart from "btnWrongPred").
btnClear = tk.Button(root, text = "Clear", command = lambda : canvas.delete("all")).pack()
btnPredict= tk.Button(root, text = "Predict", command = predictImage).pack()
btnWrongPred = tk.Button(root, text = "Wrong Prediction?", command = openPredictWindow)

# Create and display the labels (with no content).
lblRandForest = tk.Label(root)
lblKNN = tk.Label(root)
lblSVM = tk.Label(root)
lblRandForest.pack()
lblKNN.pack()
lblSVM.pack()

root.mainloop() # Run it.

## NOTE: The following code beyond this point is the pre-processing of the training data and labels.

## Convert images into a numpy array
Folders are ordered from 0 to 9 by default, so the labels can be determined based on that. <b> RUN THIS EVERYTIME NEW TRAINING IMAGES ARE ADDED. </b>

In [84]:
imgList = []
labels = []
i = 0

# Search through each image folder.
for path, dirs, files in os.walk("Data"):
    # Go through each image (from each folder) and add them to the list.
    for file in files:
        if file.endswith('.png'):
            # Reduce to a single colour depth as well (it's black and white).
            img = Image.open("Data/" + str(i) + "/" + file).convert('P', palette = Image.ADAPTIVE, colors = 2)
            img = asarray(img).reshape(-1, 1)
            imgList.append(img)
            labels.append(i)
    # Only increment if the folder isn't empty.
    # 'files' can hold empty folders for some reason...
    if len(files) > 0:
        i = i + 1

# Convert images.
n_samples = len(imgList)
npImages = asarray(imgList, dtype = "int").reshape((n_samples, -1))
np.save("trainImages", npImages)

# Convert labels.
npLabels = asarray(labels, dtype = "int")
np.save("trainLabels", npLabels)   
#print(npImages.shape)

## Testing image imports

Bits of code to experiment what the images look like.

In [ ]:
# Converts it to a single colour depth.
img = Image.open('two.png').convert('P', palette = Image.ADAPTIVE, colors = 50)
img.thumbnail((28, 28), Image.ANTIALIAS)
img.show()
npImg = asarray(img).reshape(1, -1)

pred = rf_model.predict(npImg)
print(pred)

In [ ]:
print(npImg.shape)
plt.imshow(img)
plt.show()